In [3]:
import gensim
import pandas as pd
import string
import nltk
nltk.download('omw-1.4')
nltk.download()
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\intern\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [182]:
df = pd.read_csv("train_dataset.csv")
texts = df['utterance'].apply(lambda sequence: [ltrs.lower() for ltrs in sequence if ltrs not in string.punctuation])
texts = texts.apply(lambda word: ''.join(word))
texts

0                         switch the air conditioning off
1                                         turn the ac off
2                                    switch on the aircon
3                                      turn the aircon on
4       change the minimum and maximum temperatures on...
                              ...                        
1527                   can you prove you have a conscious
1528               can you prove you are selfaware please
1529              can you prove you are self aware please
1530            can you prove you have a conscious please
1531                           prove you have a conscious
Name: utterance, Length: 1532, dtype: object

In [240]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize('self-aware', pos = 'a'))

self-aware


In [184]:
from gensim.models import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format("models/english.word2vec.bin", binary=True)

In [185]:
from nltk import pos_tag
print(pos_tag(["the"], tagset='universal'))

[('the', 'DET')]


In [247]:
model["ok"]

KeyError: "Key 'ok' not present"

In [188]:
# lemmatize_tag: n, v, a, r, s = Noun, verb, adjective, adverb, satellite adjective
# convert tag from /pos_tag/ to lemmetize_tag
def tagger(tag):
    if tag == 'ADJ':
        return 'a'
    elif tag == 'ADP':
        return 'r'
    elif tag == 'ADV':
        return 'r'
    elif tag == 'CONJ':
        return 'r'
    elif tag == 'DET':
        return 'n'
    elif tag == 'NOUN':
        return 'n'
    elif tag == 'NUM':
        return 'n'
    elif tag == 'PRT':
        return 'r'
    elif tag == 'PRON':
        return 'a'
    elif tag == 'VERB':
        return 'v'
    else: 
        return 'n/a'

In [214]:
#lemmatizing words
def word_lem(texts):
    texts_lem_df = pd.DataFrame(columns=['utterance'])
    for row in texts:
        tokens = word_tokenize(row)
        tokens = pos_tag(tokens, tagset='universal')

        sen = []
        for word in tokens:
            word = list(word)
            tag = tagger(word[1])
            if (tag != 'n/a'):
                word[0] = lemmatizer.lemmatize(word=word[0], pos=tag)
                word[1] = tag
            else:
                word[0] = word[0]
                word[1] = tag
            sen.append(word)
        df = pd.DataFrame([[sen]], columns=['utterance'])
        texts_lem_df = pd.concat([texts_lem_df, df], ignore_index=True)

    return texts_lem_df


In [219]:
#word_2_vector
_df = word_lem(texts)['utterance']
for row in _df:
    for word in row:
        tag = word[1]
        if (tag != 'n/a'):
            model[word[0]]

KeyError: "Key 'ac' not present"

In [220]:
df['utterance'] = word_lem(texts)
df

,intent,utterance
0,TurnOff,"[[switch, v], [the, n], [air, n], [condition, ..."
1,TurnOff,"[[turn, v], [the, n], [ac, n], [off, r]]"
2,TurnOn,"[[switch, n], [on, r], [the, n], [aircon, n]]"
3,TurnOn,"[[turn, v], [the, n], [aircon, n], [on, r]]"
4,ChangeTemperature,"[[change, n], [the, n], [minimum, n], [and, r]..."
...,...,...
1527,SelfAware,"[[can, v], [you, a], [prove, v], [you, a], [ha..."
1528,SelfAware,"[[can, v], [you, a], [prove, v], [you, a], [be..."
1529,SelfAware,"[[can, v], [you, a], [prove, v], [you, a], [be..."
1530,SelfAware,"[[can, v], [you, a], [prove, v], [you, a], [ha..."
